In [2]:
import torch
import torchaudio
from IPython.display import Audio, display
from speechbrain.lobes.augment import TimeDomainSpecAugment
from matplotlib import pyplot as plt

from s3prl.downstream.augment_utils.phase_perturbation import PhasePerturbation
from s3prl.downstream.mdd.dataset import L2ArcticDataset

/home/xt0r3-user/anaconda3/envs/s3prl/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/xt0r3-user/anaconda3/envs/s3prl/lib/python3.8/site-packages/speechbrain/dataio/dataio.py:26: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend(torchaudio_backend)
/home/xt0r3-user/anaconda3/envs/s3prl/lib/python3.8/site-packages/speechbrain/nnet/loss/stoi_loss.py:13: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend(torchaudio_backend)
/home/xt0r3-user/anaconda3/envs/s3prl/lib/python3.8/site-packages/speechbrain/lobes/augment.py:28: UserWarning: t

In [3]:
dataset = L2ArcticDataset(
    'train',
    '/home/xt0r3-user/cambridge/partii/dissertation/s3prl/data/l2arctic_release_v5.0',
    '/home/xt0r3-user/cambridge/partii/dissertation/s3prl/s3prl/s3prl/downstream/mdd/data/'
)

Skipping malformatted TextGrid file: /home/xt0r3-user/cambridge/partii/dissertation/s3prl/data/l2arctic_release_v5.0/YDCK/annotation/arctic_a0272.TextGrid
Skipping malformatted TextGrid file: /home/xt0r3-user/cambridge/partii/dissertation/s3prl/data/l2arctic_release_v5.0/YDCK/annotation/arctic_a0209.TextGrid


In [4]:
def play_audio(waveform, sample_rate):
  waveform = waveform.numpy()
  display(Audio(waveform, rate=sample_rate))

In [5]:
audio = torchaudio.functional.resample(dataset[0][0], orig_freq=44100, new_freq=16000)

play_audio(audio, 16000)

In [6]:
augmentation = TimeDomainSpecAugment(
    speeds=[95, 100, 105],
    sample_rate=16000,
)
play_audio(augmentation(
    torch.nn.utils.rnn.pad_sequence([audio], batch_first=True),
    torch.tensor([len(audio)], dtype=torch.long))[0], sample_rate=16000)

In [10]:
SAMPLE_RATE = 16000
n_fft = 512
n_stft = int((n_fft // 2) + 1)
phaseperturb = PhasePerturbation(
       noise_scale=0.4,
       num_stft_frequencies=2048,
       hop_length=128,
)
raw = phaseperturb([audio])

play_audio(raw[0], 16000)

In [8]:
stft = phaseperturb.apply_stft(raw[0])

stft[350:] = 0
# stft[:30] = 0

sound = phaseperturb.invert_stft(stft)


play_audio(sound, 16000)